# This notebook demonstrates a few features of the new Prolific API
Documentation: https://docs.prolific.co/docs/api-docs/public/

In [ ]:
# import libraries
import json
import urllib
import requests
import pymongo as pm
import pandas as pd

In [ ]:
class SessionWithUrlBase(requests.Session):
    def __init__(self, url_base=None, *args, **kwargs):
        super(SessionWithUrlBase, self).__init__(*args, **kwargs)
        self.url_base = url_base

    def request(self, method, url, **kwargs):
        modified_url = urllib.parse.urljoin(self.url_base, url)

        return super(SessionWithUrlBase, self).request(method, modified_url, **kwargs)

In [ ]:
# get prolific credentials
tok = pd.read_csv('prolificToken.txt', header = None).values[0][0]

s = SessionWithUrlBase(url_base='https://api.prolific.co/api/v1/')
s.headers.update({
    'Authorization': f'Token {tok}',
    'Content-Type': 'application/json'
})

In [ ]:
# creates new study on Prolific
new_study = s.post('studies/',
      data=json.dumps({
          'name': 'BACH test study',
          'internal_name': 'test_study_0',
          'description': 'I think it works.',
          'external_study_url': 'https://cogtoolslab.org:8898/index.html?' +
          'PROLIFIC_PID={{%PROLIFIC_PID%}}&STUDY_ID={{%STUDY_ID%}}&SESSION_ID={{%SESSION_ID%}}',
          'prolific_id_option': 'url_parameters',
          'completion_code': 'BACH_test_code',
          'completion_option': 'url',
          'total_available_places': 1,
          'estimated_completion_time': 10,
          'device_compatibility': ['desktop'],
          'reward': 200,
          'peripheral_requirements': [],
          'eligibility_requirements': []
      })
).json()

In [ ]:
# display information about a specific study
study_inf = s.get(f"studies/{new_study['id']}/").json()
study_inf['total_available_places']

In [ ]:
# Update study parameters. In this case, we increment study places by one
new_study = s.patch(f"studies/{new_study['id']}/",
        data= json.dumps({
            'total_available_places': new_study['total_available_places'] + 1,
        })
).json()
new_study['total_available_places']

In [ ]:
# delete study
s.delete(f"studies/{new_study['id']}/")